## How to Select how many hidden layer and neurons in a neural network

In [25]:

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU,PReLU,ELU

from sklearn.preprocessing import StandardScaler

In [28]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]



In [29]:
dataset.head(7)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0


In [30]:
#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True).astype(int)
gender=pd.get_dummies(X['Gender'],drop_first=True).astype(int)

In [31]:
X=pd.concat([X,geography,gender],axis=1)

In [32]:
X=X.drop(['Geography','Gender'],axis=1)

In [33]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [34]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [35]:
X_train.shape[1]

11

In [36]:
X_train.shape

(8000, 11)

In [37]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)

C:\Users\Mohamed Arshad\AppData\Local\Temp\ipykernel_18912\2276427309.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [38]:
model

In [39]:
layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [40]:
for i,nodes in enumerate(layers):
    print(i,nodes)

0 [20]
1 [40, 20]
2 [45, 30, 15]


In [41]:
grid_result = grid.fit(X_train, y_train)

In [42]:
[grid_result.best_score_,grid_result.best_params_]

[0.8576249957084656,
 {'activation': 'relu', 'batch_size': 256, 'epochs': 30, 'layers': [40, 20]}]

In [43]:
pred_y = grid.predict(X_test)

63/63 [==============================] - 0s 2ms/step


In [44]:
y_pred = (pred_y > 0.5)

In [45]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [46]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [47]:
cm

array([[1521,   74],
       [ 195,  210]], dtype=int64)

In [48]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)

In [49]:
score

0.8655